In [1]:
!pip install torch
!pip install torchtext
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.6 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 21.0 MB/s eta 0:00:0000:010:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
#!pip install sympy

import math
import torchtext
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from collections import Counter
from torchtext.vocab import Vocab
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer
import io
import time
import pandas as pd
import numpy as np
import pickle
import sentencepiece as spm
import matplotlib.pyplot as plt                                                 
#import sympy
from IPython.display import display, Latex

torch.manual_seed(0)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = DEVICE

In [3]:
df1 = pd.read_csv('dataset.csv')[['OUTPUT:result', 'latex']]
df2 = pd.read_csv('dataset2.csv')[['OUTPUT:result', 'latex']]
df3 = pd.read_csv('dataset3.csv')[['rus', 'latex']]
df1 = df1.rename(columns={'OUTPUT:result':'rus', 'latex': 'latex'})
df2 = df2.rename(columns={'OUTPUT:result':'rus', 'latex': 'latex'})
df = pd.concat([df1,df3],axis=0)
train = df
#

#from sklearn.model_selection import train_test_split
#train, test = train_test_split(df, test_size=0.33, random_state=42)

In [4]:
trainen = train['latex'].values.tolist()
trainja = train['rus'].values.tolist()

In [5]:
def pattern_search(s, pattern):
    k = 0
    i = 0
    shift_start = False
    res = ''
    buffer = ''
    open_bracket = False
    close_bracket = False
    
    while i < len(s):
        #print(s[i])
        if close_bracket and s[i]=='{':
            clost_bracket_index += 1
        elif close_bracket and s[i]=='}':
            close_bracket = False
        elif open_bracket and s[i]=='{':
            open_bracket = False
            clost_bracket_index = 0
            close_bracket = True
        else:
            
            if s[i]==pattern[k]:
                k+=1
                shift_start = True
                buffer += s[i]
                if k >= len(pattern):
                    shift_start = False
                    k = 0
                    buffer = ''
                    open_bracket = True
            else:
                shift_start = False
                k = 0
                res += buffer
                buffer = ''
                res += s[i]
        i+=1
    return res

def clear_latex(s):
    ar = [
        r'\mathnormal',
        r'\mathrm',
        r'\mathit',
        r'\mathbf',
        r'\mathsf',
        r'\mathtt',
        r'\mathbb',
        r'\mathfrak',
        r'\mathcal'
    ]
    
    for pattern in ar:
        s = pattern_search(s, pattern)
    
    s = s.replace('\\times','')
    return s

#s = r'\mathbb {A} _{A}(P,\alpha )^{\times }.'
#clear_latex(s)

In [6]:
%matplotlib inline
# bug - when can parse prefix correct, other chars from string with prefix became solo-chars
import matplotlib

def valid(buffer):
    if buffer == '\\': return False
    fig = plt.figure() 
    try:
        plt.text(0.03, 0.3, r"$%s$" % buffer)                                  
    except Exception as e:
        return False
    try: fig.savefig("save_file_name.pdf")
    except Exception as e: return False

    try: plt.close('all')
    except Exception as e: return False

    return True

def latex_split(eqs):
    syms = []
    buffer = ''
    
    for eq in eqs.split(' '):
        for i in eq:
            buffer += i
            if valid(buffer):
                syms += [buffer]
                buffer = ''
    return syms
            
#DF_SPLITTED = []
#for exp in latexs[:1]:
#    res = latex_split(exp)
#    DF_SPLITTED += [res]
#print(DF_SPLITTED)
#plt.close()

def simple_latex_splitter(t_raw):
    #t_raw = '\\gcd(a,b)=\\prod _{p}p^{\\min(a_{p},b_{p})}'
    res = []
    buffer = ''
    
    command = False
    for char in t_raw:
        if char in ['{','(','[']:
            res += [buffer]
            buffer = ''
            command = True
        if command:
            buffer += char
        else:
            buffer += char
        if char in ['}',')',']']:
            res += [buffer]
            buffer = ''
            command = False
    res += [buffer]
    buffer = ''
    return res

#def simple_latex_splitter(t_raw):
#    #t_raw = '\\gcd(a,b)=\\prod _{p}p^{\\min(a_{p},b_{p})}'
#    res = []
#    buffer = ''
#    
#    command = False
#    for char in t_raw:
#        if char in ['{','(','[']:
#            #res += [char]
#            res += [buffer]
#            buffer = ''
#            command = True
#            
#        if char in ['}',')',']']:
#            res += [buffer]
#            #res += [char]
#            buffer = ''
#            command = False
#            
#        if command:
#            buffer += char
#        
#    res += [buffer]
#    buffer = ''
#    return res

In [7]:
class CustomSplitter:
    def __init__(self, lang):
        self.lang = lang
        
    def encode_rus(self, sentence):
        return sentence.split(' ')
    
    def encode_latex(self, sentence):
        sentence = clear_latex(sentence)
        #sentence = latex_split(sentence)
        sentence = simple_latex_splitter(sentence)
        plt.close()
        plt.close('all')
        return sentence
    
    def encode(self, sentence, out_type):
        if self.lang == 'rus':
            return self.encode_rus(sentence)
        else: return self.encode_latex(sentence)

en_tokenizer = CustomSplitter('latex')
ja_tokenizer = CustomSplitter('rus')

In [7]:
en_tokenizer.encode('\\mathbb {A} _{A}(P,\\alpha )^{\\times }.', out_type='str')

[' A _', '{A}', '', '(P,\\alpha )', '^', '{ }', '.']

In [8]:
ja_tokenizer.encode("年金 日本に住んでいる20歳~60歳の全ての人は、公的年金制度に加入しなければなりません。", out_type='str')

['年金', '日本に住んでいる20歳~60歳の全ての人は、公的年金制度に加入しなければなりません。']

In [8]:
from tqdm import tqdm

def build_vocab(sentences, tokenizer):
    counter = Counter()
    for sentence in tqdm(sentences):
        counter.update(tokenizer.encode(sentence, out_type=str))
        plt.close('all')
    return torchtext.vocab.vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

ja_vocab = build_vocab(trainja, ja_tokenizer)
en_vocab = build_vocab(trainen, en_tokenizer)

100%|██████████| 2444/2444 [00:00<00:00, 33498.18it/s]


In [9]:
def data_process(ja, en):
    data = []
    for (raw_ja, raw_en) in tqdm(zip(ja, en)):
        ja_tensor_ = torch.tensor([ja_vocab[token] for token in ja_tokenizer.encode(raw_ja.rstrip("\n"), out_type=str)],
                            dtype=torch.long)
        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer.encode(raw_en.rstrip("\n"), out_type=str)],
                            dtype=torch.long)
        data.append((ja_tensor_, en_tensor_))
    return data


ja_vocab.set_default_index(0) # maybe not unknown
en_vocab.set_default_index(0)

train_data = data_process(trainja, trainen)

2444it [00:00, 22364.74it/s]


In [12]:
#import pickle
#
#with open('train_data2.pickle', 'wb') as handle:
#    pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
#    
#with open('ja_vocab2.pickle', 'wb') as handle:
#    pickle.dump(ja_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
#    
#with open('en_vocab2.pickle', 'wb') as handle:
#    pickle.dump(en_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)


#with open('train_data.pickle', 'rb') as handle:
#    train_data_pickle = pickle.load(handle)
#with open('ja_vocab.pickle', 'rb') as handle:
#    ja_vocab_pickle = pickle.load(handle)
#with open('en_vocab.pickle', 'rb') as handle:
#    en_vocab_pickle = pickle.load(handle)
#
#train_data_f = train_data
#ja_vocab_f = ja_vocab
#en_vocab_f = en_vocab
#
#train_data = train_data_f + train_data_pickle





#train_data_pickle

#from tqdm import tqdm
#
#def build_vocab(sentences, tokenizer):
#    counter = Counter()
#    for sentence in tqdm(sentences):
#        counter.update(tokenizer.encode(sentence, out_type=str))
#        plt.close('all')
#    return torchtext.vocab.vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
#ja_vocab = build_vocab(trainja[:10], ja_tokenizer)
#en_vocab = build_vocab(trainen[:10], en_tokenizer)
#
#def data_process(ja, en):
#    data = []
#    for (raw_ja, raw_en) in zip(ja, en):
#        ja_tensor_ = torch.tensor([ja_vocab[token] for token in ja_tokenizer.encode(raw_ja.rstrip("\n"), out_type=str)],
#                            dtype=torch.long)
#        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer.encode(raw_en.rstrip("\n"), out_type=str)],
#                            dtype=torch.long)
#        data.append((ja_tensor_, en_tensor_))
#    return data
#train_data = data_process(trainja[:10], trainen[:10])

In [10]:
BATCH_SIZE = 64
PAD_IDX = ja_vocab['<pad>']
BOS_IDX = ja_vocab['<bos>']
EOS_IDX = ja_vocab['<eos>']
def generate_batch(data_batch):
    ja_batch, en_batch = [], []
    for (ja_item, en_item) in data_batch:
        ja_batch.append(torch.cat([torch.tensor([BOS_IDX]), ja_item, torch.tensor([EOS_IDX])], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
    ja_batch = pad_sequence(ja_batch, padding_value=PAD_IDX)
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
    return ja_batch, en_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)

In [11]:
print(PAD_IDX, BOS_IDX, EOS_IDX)

1 2 3


In [12]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)


class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
    
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)
    
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [19]:
SRC_VOCAB_SIZE = len(ja_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
EMB_SIZE = 512 # 512
NHEAD = 8 # 8
FFN_HID_DIM = 512 # 512
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 3 # 3
NUM_DECODER_LAYERS = 3 # 3
NUM_EPOCHS = 64
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)
def train_epoch(model, train_iter, optimizer):
    model.train()
    losses = 0
    for idx, (src, tgt) in enumerate(train_iter):
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                                  src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
    return losses / len(train_iter)


def evaluate(model, val_iter):
    model.eval()
    losses = 0
    for idx, (src, tgt) in (enumerate(valid_iter)):
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                                  src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(val_iter)



In [20]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = time.time()
    #print('starting...')
    train_loss = train_epoch(transformer, train_iter, optimizer)
    end_time = time.time()
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, "
            f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 5.340, Epoch time = 1.279s
Epoch: 2, Train loss: 4.505, Epoch time = 1.306s
Epoch: 3, Train loss: 3.914, Epoch time = 1.430s
Epoch: 4, Train loss: 3.477, Epoch time = 1.365s
Epoch: 5, Train loss: 3.151, Epoch time = 1.307s
Epoch: 6, Train loss: 2.864, Epoch time = 1.322s
Epoch: 7, Train loss: 2.634, Epoch time = 1.216s
Epoch: 8, Train loss: 2.404, Epoch time = 1.342s
Epoch: 9, Train loss: 2.244, Epoch time = 1.263s
Epoch: 10, Train loss: 2.094, Epoch time = 1.265s
Epoch: 11, Train loss: 1.948, Epoch time = 1.323s
Epoch: 12, Train loss: 1.836, Epoch time = 1.269s
Epoch: 13, Train loss: 1.716, Epoch time = 1.304s
Epoch: 14, Train loss: 1.609, Epoch time = 1.244s
Epoch: 15, Train loss: 1.525, Epoch time = 1.276s
Epoch: 16, Train loss: 1.423, Epoch time = 1.369s
Epoch: 17, Train loss: 1.334, Epoch time = 1.398s
Epoch: 18, Train loss: 1.252, Epoch time = 1.382s
Epoch: 19, Train loss: 1.173, Epoch time = 1.345s
Epoch: 20, Train loss: 1.090, Epoch time = 1.309s
Epoch: 21

In [27]:
import pickle
from datetime import datetime

now = datetime.now() # current date and time
time_s = now.strftime("%H_%M_%S")

#torch.save(transformer, str('model' + str(time_s) + '.model'))
torch.save(transformer, 'model_production.model')

def dump_var(var):
    var_name = f'{var=}'.split('=')[0]
    with open(str(var_name + str(time_s) + '.pickle'), 'wb') as handle: pickle.dump(var, handle, protocol=pickle.HIGHEST_PROTOCOL)

def dump_last(var):
    var_name = f'{var=}'.split('=')[0]
    with open(str(var_name + '.pickle'), 'wb') as handle: pickle.dump(var, handle, protocol=pickle.HIGHEST_PROTOCOL)

#dump_var(ja_vocab)
#dump_var(en_vocab)
#
#dump_last(ja_vocab)
#dump_last(en_vocab)
#
with open('ja_vocab_production.pickle', 'wb') as handle: pickle.dump(ja_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('en_vocab_production.pickle', 'wb') as handle: pickle.dump(en_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('tokenizer_production.pickle', 'wb') as handle: pickle.dump(ja_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX: break
    return ys

def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
    model.eval()
    
    tokens = [BOS_IDX] + [src_vocab.get_stoi().get(tok,0) for tok in src_tokenizer.encode(src, out_type=str)]+ [EOS_IDX]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join([tgt_vocab.get_itos()[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

In [28]:
from IPython.display import display, Latex

transformer2 = torch.load('model_production.model')

with open('ja_vocab_production.pickle', 'rb') as handle:
    ja_vocab2 = pickle.load(handle)
    
with open('en_vocab_production.pickle', 'rb') as handle:
    en_vocab2 = pickle.load(handle)
    
with open('tokenizer_production.pickle', 'rb') as handle:
    ja_tokenizer2 = pickle.load(handle)
         
def tr(word):
    eq = translate(transformer2, word, ja_vocab2, en_vocab2, ja_tokenizer2)
    display(Latex(f'${eq}$'))
    return eq

tests_ = ["игрек плюс икс", 'предел эф от икс при икс стремящемся к альфа',
         'синус альфа синус бета', 'косинус двух', 'косинус икс', 'логарифм косинуса икс по основанию три','логарифм альфа по основанию пси']
res = ['x+y', '\lim f(x)', '\sin \alpha', '\cos 2']
#mean_bleu = 0
for key,test_ in enumerate(tests_):
    tr(test_)
#print(mean_bleu/len(tests_))

#test_ = 'синус игрек'
#tr(test_)

### эти лучшие тесты были на большом датасете из маленьких кусков на простой токенизации и вроде 64 эпохах

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
transformer, ja_vocab, en_vocab, ja_tokenizer

In [8]:
#!pip install jiwer
import nltk

#from jiwer import wer
#from jiwer import cer
#
#gleu = 0
#cer_error = 0
#error = 0
#
##bleu = 0
#tests = test.values.tolist()
#
#for sample in tests:
#    rus = sample[0]
#    true = sample[1]
#    sugg = translate(transformer, rus, ja_vocab, en_vocab, ja_tokenizer)
#    
#    #bleu_sample = nltk.translate.bleu_score.sentence_bleu([sugg], true, weights=(0.5, 0.25, 0.10, 0.05))
#    gleu_sample = nltk.translate.gleu_score.sentence_gleu([simple_latex_splitter(sugg)], simple_latex_splitter(true))
#    
#    #bleu += bleu_sample
#    gleu += gleu_sample
#    
#    ground_truth = true
#    hypothesis = sugg
#    
#    error += wer(ground_truth, hypothesis)
#    cer_error += cer(ground_truth, hypothesis)
#    
##bleu = bleu / len(tests)
#gleu = gleu / len(tests)
#error = error / len(tests)
#cer_error = cer_error / len(tests)
#print(gleu, error, cer_error)

